In [1]:
!pip install accelerate
!pip install langchain-community
!pip install langchain
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken

In [2]:
! pip install transformers trl py7zr optimum accelerate bitsandbytes
! pip install git+https://github.com/huggingface/peft
! pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118
!pip install transformers accelerate bitsandbytes -q

  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-9kzfehk9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-9kzfehk9
  Resolved https://github.com/huggingface/peft to commit f650b08abb011e4cf82b76faf33360d1a8a386f1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118


In [3]:
!pip install -U bitsandbytes

In [44]:
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM, BitsAndBytesConfig
import transformers
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda

In [5]:
model = "tiiuae/falcon-7b-instruct"

In [10]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(
    template = "Write a short message in {language} and in a {tone} tone: {message}",
    input_variables = ["tone", "message", "language"]
)
filled_prompt = prompt.format(tone = "friendly", language = "English", message = "Welcome to LangChain!")
print(filled_prompt)

Write a short message in English and in a friendly tone: Welcome to LangChain!


In [17]:
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [18]:
def text_generation(prompt):
    sequences = text_gen(
        prompt,
        max_new_tokens=100,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [19]:
text_generation(filled_prompt)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: Write a short message in English and in a friendly tone: Welcome to LangChain!
Hi! We're glad you're here. Feel free to explore our platform and connect with other language enthusiasts. We hope you have a great experience. 😊


In [41]:
def text_generation(prompt):
    sequences = text_gen(
        prompt,
        max_new_tokens=100,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    result = sequences[0]['generated_text']
    if result.startswith(prompt):
        result = result[len(prompt):].strip()
    return result

In [43]:
text_gen_chain = RunnableLambda(lambda prompt: text_generation(prompt.to_string() if hasattr(prompt, "to_string") else prompt))

prompt_title = PromptTemplate(
    template="Create a catchy tutorial title for: {message}. Use a {tone} tone.",
    input_variables=["tone", "message"],
)

prompt_desc = PromptTemplate(
    template="Write a short description for a tutorial titled '{title}' in a {tone} tone.",
    input_variables=["tone", "message", "title"]
)

output_parser = StrOutputParser()
title_chain = prompt_title | text_gen_chain | output_parser

desc_chain = (
    RunnablePassthrough.assign(title=title_chain)
    | prompt_desc
    | text_gen_chain
    | output_parser
)

inputs = {
    "tone": "friendly",
    "message": "How to use LangChain for beginners"
}

title = title_chain.invoke(inputs)
description = desc_chain.invoke(inputs)

print("Title:", title)
print("Description:", description)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Title: Beginner's Guide to LangChain: Get Started in a Few Easy Steps!
Description: Are you new to LangChain? No worries! This guide is designed to help you get up to speed quickly and unlock the power of this exciting new programming language. Let's dive right in and get started!
